In [1]:
from plantcv import plantcv as pcv
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os
from PIL import Image
from scipy import ndimage
import pandas as pd
import utils


from importlib import reload
import unionfind as UF

ModuleNotFoundError: ignored

In [ ]:
## MATLAB CODE FOR GETTING FOREGROUND AND BACKGROUND PIXEL VALUES

# ims = dir('*.jpg');

# for i = 1:size(ims,1)
#     path = ims(i).name;
#     img = imread(path);

#     foreground = impixel(img); % click on a bunch of foreground pixels
#     background = impixel(img); % then click on a bunch of background pixels

#     if i == 1
#         foreground_all = foreground;
#         background_all = background;
#     else
#         foreground_all = [foreground_all; foreground];
#         background_all = [background_all; background];
#     end
# end
#
# if size(background_all,1) > size(foreground_all,1)
#     foreground_all = foreground_all(1:size(background_all,1),:,:)
# else
#     background_all = background_all(1:size(foreground_all,1),:,:)
# end
# writematrix(background_all,'background_px.csv');
# writematrix(foreground_all,'foreground_px.csv');

In [ ]:
# This is all from the plantcv naive bayes tutorial...
# It's super hacky, but I didn't write it...
# (Which isn't to say I don't write hacky code)

# We only need to run all of this once to get the Probability Density Function files

if not os.path.isfile('pdfs.txt'):
    fg_pixels = pd.read_csv('foreground_px.csv',names=['r','g','b'])
    bg_pixels = pd.read_csv('background_px.csv',names=['r','g','b'])

    pdfs = {"hue": {}, "saturation": {}, "value": {}}

    # For each class
    for cls, cls_name in zip([fg_pixels, bg_pixels], ['fg','bg']):
        # Create a blue, green, red-formatted image ndarray with the class RGB values
        bgr_img = cv2.merge((np.asarray(cls['b'], dtype=np.uint8),
                             np.asarray(cls['g'], dtype=np.uint8),
                             np.asarray(cls['r'], dtype=np.uint8)))

        # Convert the BGR ndarray to an HSV ndarray
        hsv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)

        # Split the HSV ndarray into the component HSV channels
        hue, saturation, value = cv2.split(hsv_img)

        # Create an HSV channel dictionary that stores the channels as lists (horizontally stacked ndarrays)
        channels = {"hue": np.hstack(hue), "saturation": np.hstack(saturation), "value": np.hstack(value)}

        # For each channel
        for channel in channels:
            # Create a kernel density estimator for the channel values (Gaussian kernel)
            kde = stats.gaussian_kde(channels[channel])
            # Use the KDE to calculate a probability density function for the channel
            # Sample at each of the possible 8-bit values
            pdfs[channel][cls_name] = kde(range(0, 256))

    # Write the PDFs to a text file
    out = open('pdfs.txt', "w")
    # Write the column labels
    out.write("class\tchannel\t" + "\t".join(map(str, range(0, 256))) + "\n")
    # For each channel
    for channel, cls in pdfs.items():
        # For each class
        for class_name, pdf in cls.items():
            # Each row is the PDF for the given class and color channel
            out.write(class_name + "\t" + channel + "\t" + "\t".join(map(str, pdf)) + "\n")

In [ ]:
ims = sorted(glob('*.jpg'))
for i in range(len(ims)):
    print(i,ims[i], sep='\t')

In [ ]:
idx = 8
img = cv2.imread(ims[idx])
orig_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# resize to 20% of the original size
scale_percent = 20 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

#img3 = cv2.blur(img2)
img3 = cv2.medianBlur(img2, 15)

In [ ]:
fig, axes = plt.subplots(1,4,figsize=(16,10), sharex=True, sharey=True)
axes = np.atleast_1d(axes).ravel()

for i in range(3):
    axes[i].imshow(img3[:,:,i])
i += 1
axes[i].imshow(img3);
fig.tight_layout();

In [ ]:
# use the plantcv naive bayes classifier to find the flower pixels in the tiny blurred image
mask = pcv.naive_bayes_classifier(rgb_img=img3,pdf_file="pdfs.txt")

# Find the largest contiguous region in the foreground mask
ret, thresh = cv2.threshold(np.array(mask['fg']),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
labels = utils.get_largest_element(thresh)
labels = ndimage.binary_fill_holes(labels)

y_vals, x_vals = np.nonzero(labels)

pad = 10
y_min = np.min(y_vals) - pad
y_max = np.max(y_vals) + pad
x_min = np.min(x_vals) - pad
x_max = np.max(x_vals) + pad

segmented = img2.copy()
segmented[labels == 0, :] = 0
segmented = segmented[y_min:y_max, x_min:x_max, :]

cropped_img = img2[y_min:y_max, x_min:x_max, :]

labels = labels[y_min:y_max, x_min:x_max]
com = np.asarray(ndimage.center_of_mass(labels))

edt = np.ones_like(labels)
edt[int(com[0]), int(com[1])] = False
edt = ndimage.distance_transform_edt(edt)
edt[labels == 0] = 0
#edt = edt.astype(int)

In [ ]:
reload(UF)
pers = sorted(UF.persistence(edt),reverse=True)

In [ ]:
thr = 8
counter = 1
while pers[counter][0] > thr:
    #print(counter, pers[counter][0], sep='\t')
    counter += 1
persf = pers[:counter]
births = np.zeros((len(persf), 2), dtype=int)
for i in range(len(persf)):
    births[i] = persf[i][2]
births = births.T

deaths = np.zeros((len(persf)-1, 2), dtype=int)
for i in range(1,len(persf)):
    deaths[i-1] = persf[i][1]
deaths = deaths.T

In [ ]:
fig, axes = plt.subplots(1,4,figsize=(15,4.5), sharex=True, sharey=True)
axes = np.atleast_1d(axes).ravel()
i = 0
axes[i].imshow(cropped_img); i+=1
axes[i].imshow(segmented); i+=1
axes[i].imshow(labels); i+=1
axes[i].imshow(edt)

for i in range(len(axes)):
    axes[i].scatter([com[1]], [com[0]], color='k')
    axes[i].scatter(births[1], births[0], color='r', marker='^', s=75)
    axes[i].scatter(deaths[1], deaths[0], color='m', marker='v', s=75)

fig.suptitle(ims[idx], fontsize=20)
fig.tight_layout();

filename = ims[idx][:-4] + ' - petals'
plt.savefig(filename + '.jpg', dpi=100, bbox_inches='tight')

In [ ]:
ims[idx][:-4]

In [ ]:
for ii, i in enumerate(ims):
        img = cv2.imread(i)
        orig_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        # resize to 20% of the original size
        scale_percent = 20 # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

        # blur the heck out of it
        # TODO: Change this to a median filter
        img3 = cv2.blur(img2, (40,40))

        # use the plantcv naive bayes classifier to find the flower pixels in the tiny blurred image
        mask = pcv.naive_bayes_classifier(rgb_img=img3,pdf_file="pdfs.txt")

       # Find the largest contiguous region in the foreground mask
        ret, thresh = cv2.threshold(np.array(mask['fg']),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        connectivity = 8 
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh , connectivity , cv2.CV_32S)
        max_label, max_size = max([(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, num_labels)], key=lambda x: x[1])
        y_vals, x_vals = np.where(labels==max_label)

        # compute the rectangle that contains that region
        # TODO: Take a less tight crop
        y_min = np.min(y_vals)
        y_max = np.max(y_vals)
        x_min = np.min(x_vals)
        x_max = np.max(x_vals)

        # crop our resized image to that region
        cropped_img = img2[y_min:y_max, x_min:x_max,:]

        # convert from bgr --> rgb
        cropped_img_rgb = cv2.cvtColor(cropped_img,cv2.COLOR_BGR2RGB)
        flower_img = Image.fromarray(cropped_img_rgb)

        # use the plantcv naive bayes classifier to find the flower pixels in the tiny blurred image
        crop_bgr = cv2.cvtColor(cropped_img_rgb,cv2.COLOR_RGB2BGR)
        
        window_sz = 5
        blurred_bgr = cv2.blur(crop_bgr,(window_sz,window_sz))
        mask = pcv.naive_bayes_classifier(rgb_img=blurred_bgr,pdf_file="pdfs.txt")   
        m = mask['fg'].astype('bool')
        m2 = binary_fill_holes(m)
  
        m3 = np.zeros(([aa+6 for aa in m2.shape]))
        m3[3:-3,3:-3] = m2
        
        com = center_of_mass(m3)

        radial_filtration = RadialFiltration(center=np.array([int(round(com[1])), int(round(com[0]))]))
        im8_filtration = radial_filtration.fit_transform([m3])
        
#         im8_filtration = 256*(1-m3) + im8_filtration
#         im8_filtration = np.max(im8_filtration) - im8_filtration
        # plt.imshow(im8_filtration.squeeze())


        persistence = Persistence(homology_dimensions=[1])
        im8_cubical = persistence.fit_transform(im8_filtration)
        out = persistence.plot(im8_cubical)
        out.write_image('/Users/abby/Desktop/out.jpg')

        persistence_img = Image.open('/Users/abby/Desktop/out.jpg')

        diagramFiltering = Filtering(epsilon=15, homology_dimensions=([1]))
        x_filtered = diagramFiltering.fit_transform(im8_cubical)

        num_petals = len(x_filtered[0])
        
        # Create a figure with four subplots arranged horizontally
        fig, axes = plt.subplots(1, 5, figsize=(15, 3))

        # Turn off the axis labels for each subplot
        for ax in axes:
            ax.axis("off")

        # Display each image in its corresponding subplot, with a title
        axes[0].imshow(orig_rgb)
        axes[0].set_title("Original Image")

        axes[1].imshow(flower_img)
        axes[1].set_title("Cropped Flower from Naive Bayes")

        axes[2].imshow(m2)
        axes[2].set_title("Flower Mask")

        axes[3].imshow(im8_filtration.squeeze())
        axes[3].set_title("Radial Filtration")

        persistence_img2 = np.array(persistence_img)[90:-55,45:-75]
        axes[4].imshow(persistence_img2)
        axes[4].set_title("Persistence Diagram")


        # Add an overall title for the figure
        fig.suptitle("Predicted Number of Petals: "+str(num_petals),y=1.05, fontsize=14)

        # Display the figure
        plt.show()
        
        break

In [ ]:
import numpy as np
from ripser import ripser
from persim import plot_diagrams

diagrams = ripser(np.squeeze(im8_filtration))['dgms']
plot_diagrams(diagrams, show=True)

In [ ]:
im8_filtration.shape